# Preprocessing arbitrarily structured data for AI with Awkward Array

Usually the arrays you deal with are rectangular (in $n$ dimensions; "rectilinear").

<center>
<img src="img/8-layer_cube.jpg" width="50%">
</center>

What if we had data like this?

```json
[
  [[1.84, 0.324]],
  [[-1.609, -0.713, 0.005], [0.953, -0.993, 0.011, 0.718]],
  [[0.459, -1.517, 1.545], [0.33, 0.292]],
  [[-0.376, -1.46, -0.206], [0.65, 1.278]],
  [[], [], [1.617]],
  []
]
[
  [[-0.106, 0.611]],
  [[0.118, -1.788, 0.794, 0.658], [-0.105]]
]
[
  [[-0.384], [0.697, -0.856]],
  [[0.778, 0.023, -1.455, -2.289], [-0.67], [1.153, -1.669, 0.305, 1.517, -0.292]]
]
[
  [[0.205, -0.355], [-0.265], [1.042]],
  [[-0.004], [-1.167, -0.054, 0.726, 0.213]],
  [[1.741, -0.199, 0.827]]
]
```

Or **heterogenous** data like this?

```json
[
  {"fill": "#b1b1b1", "stroke": "none", "points": [{"x": 5.27453, "y": 1.03276},
    {"x": -3.51280, "y": 1.74849}]},
  {"fill": "#b1b1b1", "stroke": "none", "points": [{"x": 8.21630, "y": 4.07844},
    {"x": -0.79157, "y": 3.49478}, {"x": 16.38932, "y": 5.29399},
    {"x": 10.38641, "y": 0.10832}, {"x": -2.07070, "y": 14.07140},
    {"x": 9.57021, "y": -0.94823}, {"x": 1.97332, "y": 3.62380},
    {"x": 5.66760, "y": 11.38001}, {"x": 0.25497, "y": 3.39276},
    {"x": 3.86585, "y": 6.22051}, {"x": -0.67393, "y": 2.20572}]},
  {"fill": "#d0d0ff", "stroke": "none", "points": [{"x": 3.59528, "y": 7.37191},
    {"x": 0.59192, "y": 2.91503}, {"x": 4.02932, "y": -1.13601},
    {"x": -1.01593, "y": 1.95894}, {"x": 1.03666, "y": 0.05251}]},
  {"fill": "#d0d0ff", "stroke": "none", "points": [{"x": -8.78510, "y": -0.00497},
    {"x": -15.22688, "y": 3.90244}, {"x": 5.74593, "y": 4.12718}]},
  {"fill": "none", "stroke": "#000000", "points": [{"x": 4.40625, "y": -6.953125},
    {"x": 4.34375, "y": -7.09375}, {"x": 4.3125, "y": -7.140625},
    {"x": 4.140625, "y": -7.140625}]},
  {"fill": "none", "stroke": "#808080", "points": [{"x": 0.46875, "y": -0.09375},
    {"x": 0.46875, "y": -0.078125}, {"x": 0.46875, "y": 0.53125}]}
]
```

It might be possible to turn these datasets into tabular form using surrogate keys and database normalization, but

 * they could be inconvenient or less efficient in that form, depending on what we want to do,
 * they were very likely _given_ in a ragged/untidy form. You can't ignore the data-cleaning step!

_VK: do you mean padding here?_

<br>

Dealing with these datasets as JSON or Python objects is inefficient for the same reason as for lists of numbers.

<br>

We want arbitrary data structure with array-oriented interface and performance...

<center>
<img src="img/awkward-motivation-venn-diagram.svg" width="40%">
</center>

## Libraries for irregular arrays

<br>

<table>
<tr style="background: white;"><td width="35%"><img src="img/logo-arrow.svg" width="100%"></td><td style="padding-left: 50px;">In-memory format and an ecosystem of tools, an "exploded database" (database functionality provided as interchangeable pieces). Strong focus on delivering data, zero-copy, between processes.</td></tr>
<tr style="background: white; height: 30px;"><td></td><td></td></tr>
<tr style="background: white;"><td width="35%"><img src="img/logo-awkward.svg" width="100%"></td><td style="padding-left: 50px;">Library for array-oriented programming like NumPy, but for arbitrary data structures. Losslessly zero-copy convertible to/from Arrow and Parquet.</td></tr>
<tr style="background: white; height: 30px;"><td></td><td></td></tr>
<tr style="background: white;"><td width="35%"><img src="img/logo-parquet.svg" width="100%"></td><td style="padding-left: 50px;">Disk format for storing large datasets and (selectively) retrieving them.</td></tr>
</table>

<img src="../img/logo-arrow.svg" width="30%">

<br>

In [1]:
import pyarrow as pa

<br>

In [2]:
arrow_array = pa.array([
    [{"x": 1.1, "y": [1]}, {"x": 2.2, "y": [1, 2]}, {"x": 3.3, "y": [1, 2, 3]}],
    [],
    [{"x": 4.4, "y": [1, 2, 3, 4]}, {"x": 5.5, "y": [1, 2, 3, 4, 5]}]
])

<br>

In [3]:
arrow_array.type

ListType(list<item: struct<x: double, y: list<item: int64>>>)

<br>

In [4]:
arrow_array

[
  -- is_valid: all not null
  -- child 0 type: double
    [
      1.1,
      2.2,
      3.3
    ]
  -- child 1 type: list<item: int64>
    [
      [
        1
      ],
      [
        1,
        2
      ],
      [
        1,
        2,
        3
      ]
    ],
  -- is_valid: all not null
  -- child 0 type: double
    []
  -- child 1 type: list<item: int64>
    [],
  -- is_valid: all not null
  -- child 0 type: double
    [
      4.4,
      5.5
    ]
  -- child 1 type: list<item: int64>
    [
      [
        1,
        2,
        3,
        4
      ],
      [
        1,
        2,
        3,
        4,
        5
      ]
    ]
]

<img src="img/logo-awkward.svg" width="30%">

<br>

In [5]:
import awkward as ak

<br>

In [6]:
awkward_array = ak.from_arrow(arrow_array)
awkward_array

<Array [[{x: 1.1, y: [1]}, ..., {...}], ...] type='3 * var * ?{x: ?float64,...'>

<img src="img/logo-parquet.svg" width="30%">

<br>

In [7]:
ak.to_parquet(awkward_array, "/tmp/file.parquet")

  created_by: parquet-cpp-arrow version 11.0.0
  num_columns: 2
  num_rows: 3
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 0

<br>

In [8]:
ak.from_parquet("/tmp/file.parquet")

<Array [[{x: 1.1, y: [1]}, ..., {...}], ...] type='3 * var * ?{x: ?float64,...'>

## Awkward Array

In [9]:
ragged = ak.Array([
    [
      [[1.84, 0.324]],
      [[-1.609, -0.713, 0.005], [0.953, -0.993, 0.011, 0.718]],
      [[0.459, -1.517, 1.545], [0.33, 0.292]],
      [[-0.376, -1.46, -0.206], [0.65, 1.278]],
      [[], [], [1.617]],
      []
    ],
    [
      [[-0.106, 0.611]],
      [[0.118, -1.788, 0.794, 0.658], [-0.105]]
    ],
    [
      [[-0.384], [0.697, -0.856]],
      [[0.778, 0.023, -1.455, -2.289], [-0.67], [1.153, -1.669, 0.305, 1.517, -0.292]]
    ],
    [
      [[0.205, -0.355], [-0.265], [1.042]],
      [[-0.004], [-1.167, -0.054, 0.726, 0.213]],
      [[1.741, -0.199, 0.827]]
    ]
])

**Multidimensional indexing**

In [10]:
ragged[3, 1, -1, 2]

0.726

<br>

**Basic slicing**

In [11]:
ragged[3, 1:, -1, 1:3]

<Array [[-0.054, 0.726], [-0.199, 0.827]] type='2 * var * float64'>

**Awkward slicing**

In [12]:
ragged > 0

<Array [[[[True, True]], [...], ..., []], ...] type='4 * var * var * var * ...'>

<br>

In [13]:
ragged[ragged > 0]

<Array [[[[1.84, 0.324]], [...], ..., []], ...] type='4 * var * var * var *...'>

**Reductions**

In [14]:
ak.sum(ragged)

2.8980000000000006

<br>

In [15]:
ak.sum(ragged, axis=-1)

<Array [[[2.16], [-2.32, ...], ..., []], ...] type='4 * var * var * float64'>

<br>

In [16]:
ak.sum(ragged, axis=0)

<Array [[[1.56, 0.58], [...], [1.04]], ..., []] type='6 * var * var * float64'>

How are reductions even defined for ragged arrays?

<center>
<img src="img/example-reduction-sum.svg" width="50%">
</center>

In [17]:
small_ragged = ak.Array([[   1, 2, 4],
                         [          ],
                         [None, 8,  ],
                         [  16      ]])

<br>

In [18]:
ak.sum(small_ragged, axis=0)

<Array [17, 10, 4] type='3 * int64'>

## Heterogenous data

In [19]:
structured = ak.Array([
  {"fill": "#b1b1b1", "stroke": "none", "points": [{"x": 5.27453, "y": 1.03276},
    {"x": -3.51280, "y": 1.74849}]},
  {"fill": "#b1b1b1", "stroke": "none", "points": [{"x": 8.21630, "y": 4.07844},
    {"x": -0.79157, "y": 3.49478}, {"x": 16.38932, "y": 5.29399},
    {"x": 10.38641, "y": 0.10832}, {"x": -2.07070, "y": 14.07140},
    {"x": 9.57021, "y": -0.94823}, {"x": 1.97332, "y": 3.62380},
    {"x": 5.66760, "y": 11.38001}, {"x": 0.25497, "y": 3.39276},
    {"x": 3.86585, "y": 6.22051}, {"x": -0.67393, "y": 2.20572}]},
  {"fill": "#d0d0ff", "stroke": "none", "points": [{"x": 3.59528, "y": 7.37191},
    {"x": 0.59192, "y": 2.91503}, {"x": 4.02932, "y": -1.13601},
    {"x": -1.01593, "y": 1.95894}, {"x": 1.03666, "y": 0.05251}]},
  {"fill": "#d0d0ff", "stroke": "none", "points": [{"x": -8.78510, "y": -0.00497},
    {"x": -15.22688, "y": 3.90244}, {"x": 5.74593, "y": 4.12718}]},
  {"fill": "none", "stroke": "#000000", "points": [{"x": 4.40625, "y": -6.953125},
    {"x": 4.34375, "y": -7.09375}, {"x": 4.3125, "y": -7.140625},
    {"x": 4.140625, "y": -7.140625}]},
  {"fill": "none", "stroke": "#808080", "points": [{"x": 0.46875, "y": -0.09375},
    {"x": 0.46875, "y": -0.078125}, {"x": 0.46875, "y": 0.53125}]}
])

<br>

In [20]:
import numpy as np

**Elementwise formulas**

In [21]:
np.sqrt(structured["points", "x"]**2 + structured["points", "y"]**2)

<Array [[5.37, 3.92], ..., [0.478, 0.475, 0.708]] type='6 * var * float64'>

Alternative (and maybe more convenient) formulation

<br>

In [22]:
np.sqrt(structured.points.x**2 + structured.points.y**2)

<Array [[5.37, 3.92], ..., [0.478, 0.475, 0.708]] type='6 * var * float64'>

Calculate the length of each path.

The length of a given path is the sum of segment lengths, defined between `points`, $\displaystyle \sum_i^{n - 1} \Delta r_i$, where

   * the segment length $\Delta r_i = \sqrt{\Delta x_i^2 + \Delta y_i^2}$

<br>

<center>
<img src="img/length-by-segment.svg" width="50%">
</center>

In [23]:
np.sum(np.sqrt((structured.points.x[:, 1:] - structured.points.x[:, 0:-1])**2 + (structured.points.y[:, 1:] - structured.points.y[:, 0:-1])**2), axis=-1)

<Array [8.82, 110, 19.4, 28.5, 0.382, 0.625] type='6 * float64'>

## Bonus: Combinatorics

Some operations are more meaningful on irregular arrays than rectilinear ones:

<table style="width: 60%">
<tr style="background: white; padding-top: 0px;"><td width="50%"><img src="../img/cartoon-cartesian.svg" width="100%"></td><td width="50%"><img src="../img/cartoon-combinations.svg" width="100%"></td></tr>
</table>

[ak.cartesian](https://awkward-array.org/doc/main/reference/generated/ak.cartesian.html) takes a [Cartesian product](https://en.wikipedia.org/wiki/Cartesian_product) of lists from $n$ different arrays, making an array of lists of $n$-tuples.

[ak.combinations](https://awkward-array.org/doc/main/reference/generated/ak.combinations.html) takes $n$ [samples without replacement](http://prob140.org/sp18/textbook/notebooks-md/5_04_Sampling_Without_Replacement.html) of lists from a single array, making an array of lists of $n$-tuples.

<center>
<img src="img/cartoon-cartesian.svg" width="30%">
</center>

In [24]:
numbers = ak.Array([[1, 2, 3], [], [4]])
letters = ak.Array([["a", "b"], ["c"], ["d", "e"]])

<br>

In [25]:
ak.cartesian([numbers, letters])

<Array [[(1, 'a'), (...), ..., (3, 'b')], ...] type='3 * var * (int64, string)'>

<center>
<img src="img/cartoon-combinations.svg" width="30%">
</center>

In [26]:
values = ak.Array([[1.1, 2.2, 3.3, 4.4], [], [5.5, 6.6]])

<br>

In [27]:
ak.combinations(values, 2)

<Array [[(1.1, 2.2), ..., (3.3, ...)], ...] type='3 * var * (float64, float64)'>

## GPU Capabilities

In [28]:
# take the exmple ragged array from above
array_cpu = ragged
ak.backend(array_cpu)

'cpu'

In [29]:
array_gpu = ak.to_backend(array_cpu, "cuda")
ak.backend(array_gpu)

'cuda'

In [30]:
# repeat the same example as above but on the divice
structured_gpu = ak.to_backend(structured, "cuda")
array_sum = np.sum(np.sqrt((structured_gpu.points.x[:, 1:] - structured_gpu.points.x[:, 0:-1])**2 + (structured_gpu.points.y[:, 1:] - structured_gpu.points.y[:, 0:-1])**2), axis=-1)

# display
display(array_sum[array_sum > 10]) # selection/indexing
display(array_sum[array_sum < 10]) # selection/indexing

# everything stays in the device
ak.backend(array_sum)

<Array [109.70188358249592, ..., 28.508228222215624] type='3 * float64'>

<Array [8.816430000958439, 0.382100141832189, 0.625] type='3 * float64'>

'cuda'